# 🧠 Projeto: Impacto dos Alojamentos Locais no Mercado da Habitação
Este notebook faz parte da Fase 1 do projeto da unidade curricular.

## 📌 Objetivos:
- Integrar dados de diferentes fontes.
- Analisar a relação entre densidade de alojamentos locais, população e rendas
- Implementar técnicas de data cleaning, schema integration e identity resolution

---

## 📦 Importação de Dados

Nesta secção, são importados os diferentes datasets necessários para a análise do impacto do Airbnb em fatores urbanos em Portugal. Cada dataset provém de uma fonte distinta e representa uma perspetiva específica da realidade urbana, como densidade populacional, oferta de alojamentos locais, preços de renda, e dados territoriais.

Os datasets importados são os seguintes:

- **`portugal_listings.csv`**: Contém informações sobre os imóveis disponíveis para aluguer em território nacional, com detalhes como `Price`, `District`, `City` e `Town`.
- **`rendasm2.json`**: Fornece os valores médios das rendas por metro quadrado em diferentes regiões para o ano de **2023**.
- **`densidadePopulacional.json`**: Indica a densidade populacional (Nº habitantes/km²) por concelho para o ano de **2023**.
- **`densidadealojamentosm2.json`**: Apresenta a densidade de alojamentos locais (Nº alojamentos/km²), com granularidade ao nível da freguesia, referente ao ano de **2021**.
- **`Areas_Freg_Conc_Dist_Pais_CAOP2019.xls`**: Ficheiro do CAOP (Carta Administrativa Oficial de Portugal) que define a estrutura administrativa do país, incluindo distritos, concelhos e freguesias. Utilizado como base de referência para unificar códigos geográficos (`DICO`, `geocod`, etc.).

> ⚠️ Cada ficheiro foi lido respeitando a estrutura interna dos formatos `.csv`, `.json` ou `.xls`, e apenas os dados relevantes foram extraídos para posterior limpeza e integração.



In [154]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Leitura dos datasets
# O dataset listings contém informações sobre os imóveis disponíveis para venda
listings = pd.read_csv(r'dados\portugal_listings.csv')

# O dataset rendas contém informações sobre os preços das rendas por metro quadrado
rendas = pd.read_json(r'dados\rendasm2.json')
rendas = pd.DataFrame(rendas['Dados'][0]['2023'])

# O dataset densidade_pop contém informações sobre a densidade populacional (Nº/km2) por conselho
densidade_pop = pd.read_json(r'dados\densidadePopulacional.json')
densidade_pop = pd.DataFrame(densidade_pop['Dados'][0]['2023'])

# O dataset densidade_aloj contém informações sobre a densidade de alojamentos (Nº/km2) por freguesia
densidade_aloj = pd.read_json(r'dados\densidadealojamentosm2.json')
densidade_aloj = pd.DataFrame(densidade_aloj['Dados'][0]['2021'])

# A tabela municipios contém informações sobre a ordenação territorial portuguesa
concelhos = pd.read_excel(r'dados\Areas_Freg_Conc_Dist_Pais_CAOP2019.xls', sheet_name='Areas_Concelhos_CAOP2019')

Após a importação dos dados, o próximo passo será o data profiling e integration, de modo a perceber as estruturas de dados

---

## 🔍 Data Profiling

Após a importação dos dados, foi realizada uma análise exploratória (data profiling) para compreender a estrutura de cada dataset. Esta análise incluiu:

- Verificação dos tipos de dados;
- Número de valores nulos;
- Percentagem de nulos;
- Cardinalidade (valores únicos) por coluna.

A função usada para este processo foi:


In [155]:
def DataProfiling(dataframe: pd.DataFrame):
    # Data profiling completo para todas as colunas
    data_profiling = pd.DataFrame({
        "Coluna": dataframe.columns,
        "Tipo de Dado": dataframe.dtypes.values,
        "Valores Não Nulos": dataframe.notnull().sum().values,
        "Valores Nulos": dataframe.isnull().sum().values,
        "% Nulos": (dataframe.isnull().mean() * 100).round(2).values,
        "Valores Únicos": dataframe.nunique().values
    })
    display(data_profiling)  

### Areas_Freg_Conc_Dist_Pais_CAOP2019.xls



In [156]:
DataProfiling(concelhos)

,Coluna,Tipo de Dado,Valores Não Nulos,Valores Nulos,% Nulos,Valores Únicos
0,DICO,int64,308,0,0.0,308
1,NUTSI_DSG,object,308,0,0.0,3
2,NUTSI_COD,int64,308,0,0.0,3
3,NUTSII_DSG,object,308,0,0.0,7
4,NUTSII_COD,int64,308,0,0.0,7
5,NUTSIII_DSG,object,308,0,0.0,25
6,NUTSIII_COD,object,308,0,0.0,25
7,DISTRITO_ILHA_DSG,object,308,0,0.0,29
8,CONCELHO_DSG,object,308,0,0.0,307
9,AREA_2019_ha,float64,308,0,0.0,308


⚙️ **Seleção de Colunas Relevantes — `Areas_Concelhos_CAOP2019`**

Contém 14 colunas, das quais apenas 3 foram consideradas relevantes:

- `DICO`: Identificador único do concelho (**manter**).
- `CONCELHO_DSG`: Nome oficial do concelho (**manter**).
- `NUTSIII_DSG`: Região NUTS III (**manter**).

As colunas a eliminar incluem:
- `NUTSI_DSG`, `NUTSI_COD`, `NUTSII_DSG`, `NUTSII_COD`, `NUTSIII_COD`: não necessárias à análise.
- `DISTRITO_ILHA_DSG`: nível distrital, não será utilizado.
- `AREA_2019_ha`, `AREA_2019_km2`, `PERIM_km`, `ALTITUDE_MAX_m`, `ALTITUDE_MIN_m`: não relevantes para o estudo.



### densidadealojamentosm2.json



In [157]:
DataProfiling(densidade_aloj)

,Coluna,Tipo de Dado,Valores Não Nulos,Valores Nulos,% Nulos,Valores Únicos
0,geocod,object,3439,0,0.0,3439
1,geodsg,object,3439,0,0.0,3075
2,ind_string,object,3439,0,0.0,1656
3,valor,object,3439,0,0.0,1656


⚙️ **Seleção de Colunas Relevantes — `densidadealojamentosm2`**

Contém 4 colunas, sendo úteis:

- `geocod`: Identificador único da freguesia (**manter**).
- `geodsg`: Nome oficial da freguesia (**manter**).
- `valor`: Valor do indicador (**manter**, a converter para `float`).

A coluna `ind_string` é redundante e será **eliminada**.

### DensidadePopulacional.json



In [158]:
DataProfiling(densidade_pop)

,Coluna,Tipo de Dado,Valores Não Nulos,Valores Nulos,% Nulos,Valores Únicos
0,geocod,object,347,0,0.0,347
1,geodsg,object,347,0,0.0,340
2,ind_string,object,347,0,0.0,316
3,valor,object,347,0,0.0,316


⚙️ **Seleção de Colunas Relevantes — `DensidadePopulacional`**

Com estrutura semelhante ao anterior. Colunas relevantes:

- `geocod`: Identificador do concelho (**manter**).
- `geodsg`: Nome oficial do concelho (**manter**).
- `valor`: Valor do indicador (**manter**, a converter para `float`).

`ind_string` será **eliminado**.




### portugal_listings.csv

In [159]:
DataProfiling(listings)

,Coluna,Tipo de Dado,Valores Não Nulos,Valores Nulos,% Nulos,Valores Únicos
0,Price,float64,135236,300,0.22,4754
1,District,object,135536,0,0.00,27
2,City,object,135536,0,0.00,275
3,Town,object,135534,2,0.00,2263
4,Unnamed: 4,float64,0,135536,100.00,0
5,Unnamed: 5,float64,0,135536,100.00,0
6,Unnamed: 6,float64,0,135536,100.00,0
7,Unnamed: 7,float64,0,135536,100.00,0
8,Unnamed: 8,float64,0,135536,100.00,0
9,Unnamed: 9,float64,0,135536,100.00,0


⚙️ **Seleção de Colunas Relevantes — `portugal_listings`**

Contém 14 colunas. Apenas 3 foram consideradas essenciais:

- `Price`: Preço do imóvel (**manter**).
- `District`: Nome do distrito (**manter**).
- `City`: Nome da cidade (**manter**).

As colunas `Town` e `Unnamed: 4` a `Unnamed: 13` serão **eliminadas** por irrelevância e redundância, respetivamente.



### rendasm2.json


In [160]:
DataProfiling(rendas)

,Coluna,Tipo de Dado,Valores Não Nulos,Valores Nulos,% Nulos,Valores Únicos
0,geocod,object,638,0,0.00,638
1,geodsg,object,638,0,0.00,616
2,ind_string,object,638,0,0.00,343
3,valor,object,431,207,32.45,341
4,sinal_conv,object,207,431,67.55,2
5,sinal_conv_desc,object,207,431,67.55,2


⚙️ **Seleção de Colunas Relevantes — `rendasm2`**

Contém 6 colunas. As seguintes foram mantidas:

- `geocod`: Identificador do concelho (**manter**).
- `geodsg`: Nome do concelho (**manter**).
- `valor`: Valor médio da renda (**manter**, a converter para `float`).

As colunas `ind_string`, `sinal_conv`, `sinal_conv_desc` foram consideradas auxiliares e serão **eliminadas**.



### 📊 Conclusões do Data Profiling

Após a leitura e análise dos datasets, foram identificadas as seguintes características e decisões de limpeza e preparação:


#### 📌 Colunas-Chave Identificadas

| Dataset                     | Chave Primária                              | Chave(s) Secundária(s)         |
|----------------------------|---------------------------------------------|--------------------------------|
| `Areas_Concelhos_CAOP2019` | `DICO` → renomeado para `MunicipalityCode` | `CONCELHO_DSG`, `NUTSIII_DSG` |
| `densidadePopulacional`    | `geocod` (últimos 4 dígitos) → `MunicipalityCode` | `geodsg`             |
| `densidadealojamentosm2`   | `geocod` (primeiros 4 dígitos) → `MunicipalityCode` | `geodsg`            |
| `rendasm2`                 | `geocod` (últimos 4 dígitos) → `MunicipalityCode` | `geodsg`             |
| `portugal_listings`        | `City`                                      | `District`, `Price`             |

⚠️ Nota: `geocod` foi mapeado para `MunicipalityCode` com `str[:4]` ou `str[-4:]` conforme o dataset, garantindo uniformidade nas junções.


#### 🔍 Conversões e Limpezas Realizadas

- As colunas `valor` foram convertidas para `float` com `pd.to_numeric(errors='coerce')`.
- Os registos com valores **nulos em `Price`** (no dataset `listings`) foram **removidos**.
- Os valores nulos nas colunas `valor` dos datasets **`rendas`** foram **substituídos pela média do distrito**.
- Todas as colunas geográficas foram **normalizadas**:
  - Remoção de acentos;
  - Conversão para minúsculas;
  - Remoção de espaços em branco;
  - Criação de colunas auxiliares como `*_norm`.


#### 🧱 Diferença de Granularidade

- Os datasets `densidadealojamentosm2` e `rendasm2` estão ao nível da **freguesia**.
- Os restantes datasets estão ao nível do **concelho** (`MunicipalityCode`).

🔄 **Solução**:
- Realizar um **agrupamento por concelho** (usando os primeiros ou últimos 4 dígitos de `geocod`).
- As **médias por concelho** serão calculadas para:
  - Densidade de alojamentos locais;
  - Preço das rendas por m².
- Isso garante coerência na granularidade entre os diferentes conjuntos de dados para posterior integração e análise.


---

## 🧽 Data Cleaning & Preparation

Após a fase de **Data Profiling**, foi possível identificar várias inconsistências nos datasets utilizados neste projeto. Esta fase de **Data Cleaning & Integration** tem como objetivo preparar os dados para a integração e análise final, assegurando que todas as fontes de dados partilham uma estrutura coerente e limpa.


### 🔍 Problemas Identificados

- Colunas com valores numéricos armazenados como `string`;
- Utilização inconsistente de separadores decimais (`"."` e `","`);
- Existência de **valores nulos** em campos críticos como `Price` e `valor`;
- Presença de acentos, letras maiúsculas/minúsculas e espaços inconsistentes em nomes geográficos (`District`, `Town`, `City`, `geodsg`, etc.);
- Diferença de granularidade entre datasets (ex: freguesia vs concelho).



### 🎯 Objetivos da Limpeza e Preparação

1. **Conversão de colunas numéricas** para o tipo `float`, garantindo a coerência dos dados;
2. **Tratamento de valores nulos**, através de:
   - Remoção em colunas como `Price`;
   - Substituição pela **média distrital** no dataset com `valor`, `rendasm2`;
3. **Normalização dos nomes de localidades** para garantir a consistência entre datasets:
   - Remoção de acentos;
   - Conversão para minúsculas;
   - Eliminação de espaços em branco;
4. **Criação de colunas normalizadas (`*_norm`)**, que servirão como **chaves secundárias** durante a integração dos dados;
5. **Harmonização das chaves primárias**, como `MunicipalityCode`, obtidas a partir do `geocod` (início ou fim da string, conforme o dataset);
6. **Preparação de DataFrames tratados**, com apenas as colunas essenciais para futura análise.


Com estas operações de limpeza, os datasets ficam prontos para a próxima etapa do projeto: **Schema Integration e Identity Resolution**.


In [161]:
!pip install unidecode


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [162]:
import unidecode

# Normalizar nomes
def normalizar(texto):
    return unidecode.unidecode(str(texto).strip().lower())

#### Areas_Freg_Conc_Dist_Pais_CAOP2019

In [163]:
dfConcelhos = pd.DataFrame(concelhos)

# ✅ Selecionar colunas relevantes
colunasParaManter = ['DICO', 'CONCELHO_DSG', 'NUTSIII_DSG']
dfConcelhos = dfConcelhos[colunasParaManter]

# 🔁 Renomear colunas para nomes mais descritivos
dfConcelhos = dfConcelhos.rename(columns={
    'DICO': 'MunicipalityCode',
    'CONCELHO_DSG': 'Municipality',
    'NUTSIII_DSG': 'Region'
})

# 🧽 Normalizar colunas de texto
dfConcelhos['MunicipalityCode'] = dfConcelhos['MunicipalityCode'].astype(str).str.zfill(4)
dfConcelhos['Municipality_norm'] = dfConcelhos['Municipality'].apply(normalizar)
dfConcelhos['Region_norm'] = dfConcelhos['Region'].apply(normalizar)

# 💾 Exportar tabela final com as colunas normalizadas incluídas
dfConcelhosTratado = dfConcelhos[[
    'MunicipalityCode', 'Municipality', 'Municipality_norm',
    'Region', 'Region_norm'
]].drop_duplicates().copy()

dfConcelhosTratado.to_csv(r'dados\dados tradados/concelhosTratado.csv', index=False)


#### densidadealojamentosm2

In [164]:
# 📥 Criar DataFrame a partir do dataset original
dfAloj = pd.DataFrame(densidade_aloj)

# 🧾 Extrair código do concelho (4 primeiros dígitos do geocod)
dfAloj['MunicipalityCode'] = dfAloj['geocod'].str[:4]

# 🔤 Normalizar nome da freguesia
dfAloj['FreguesiaNorm'] = dfAloj['geodsg'].apply(normalizar)

# 🔁 Juntar com concelhos para obter nomes e norm
dfAlojMerged = dfAloj.merge(
    dfConcelhos[['MunicipalityCode', 'Municipality', 'Municipality_norm']],
    on='MunicipalityCode', how='left'
)

# ✅ Filtrar apenas matches válidos
alojamentosMatch = dfAlojMerged[dfAlojMerged['Municipality'].notna()].copy()

# 🔢 Converter densidade para float
alojamentosMatch['Density'] = pd.to_numeric(alojamentosMatch['valor'], errors='coerce')

# 🔄 Substituir valores nulos pela média do concelho
alojamentosMatch['Density'] = alojamentosMatch.groupby('MunicipalityCode')['Density'].transform(
    lambda x: x.fillna(x.mean())
)

# 📊 Agregar média da densidade por concelho
densidadeAlojMedia = (
    alojamentosMatch
    .groupby(['MunicipalityCode', 'Municipality', 'Municipality_norm'])
    .agg({'Density': 'mean'})
    .reset_index()
    .rename(columns={'Density': 'DensityAlojMean'})
)

# 💾 Exportar CSV final
densidadeAlojMedia.to_csv(r'dados\dados tradados/densidadeAlojTratado.csv', index=False)


#### DensidadePopulacional

In [165]:
# 📥 Criar DataFrame a partir do dataset original
dfPop = pd.DataFrame(densidade_pop)

# 🧾 Extrair código do concelho (últimos 4 dígitos do geocod)
dfPop['MunicipalityCode'] = dfPop['geocod'].str[-4:]

# 🔤 Normalizar nome do concelho (geodsg → nome da unidade geográfica)
dfPop['MunicipalityNorm'] = dfPop['geodsg'].apply(normalizar)

# 🔁 Juntar com concelhos para obter nomes oficiais
dfPopMerged = dfPop.merge(
    dfConcelhos[['MunicipalityCode', 'Municipality', 'Municipality_norm']],
    on='MunicipalityCode', how='left'
)

# ✅ Filtrar apenas matches válidos
populacionalMatch = dfPopMerged[dfPopMerged['Municipality'].notna()].copy()

# 🔢 Converter densidade para float
populacionalMatch['Density'] = pd.to_numeric(populacionalMatch['valor'], errors='coerce')

# 🔄 Substituir valores nulos pela média do concelho
populacionalMatch['Density'] = populacionalMatch.groupby('MunicipalityCode')['Density'].transform(
    lambda x: x.fillna(x.mean())
)

# 📊 Agregar média da densidade por concelho
densidadePopMedia = (
    populacionalMatch
    .groupby(['MunicipalityCode', 'Municipality', 'Municipality_norm'])
    .agg({'Density': 'mean'})
    .reset_index()
    .rename(columns={'Density': 'DensityPopMean'})
)

# 💾 Exportar CSV final
densidadePopMedia.to_csv(r'dados\dados tradados/densidadePopTratado.csv', index=False)


#### portugal_listings

In [166]:
# 📥 Criar DataFrame a partir do CSV original
dfListings = pd.DataFrame(listings)

# 🧹 Remover colunas 'Unnamed' e 'Town' (não será usada)
dfListings = dfListings.drop(columns=[col for col in dfListings.columns if "Unnamed" in col])
dfListings = dfListings.drop(columns=['Town'], errors='ignore')


# 💰 Converter 'Price' para float
dfListings['Price'] = pd.to_numeric(dfListings['Price'], errors='coerce')

# ✅ Filtrar linhas com preço válido (> 0)
dfListings = dfListings[dfListings['Price'].notna() & (dfListings['Price'] > 0)].copy()

# 🔤 Normalizar nomes de localização
dfListings['District_norm'] = dfListings['District'].apply(normalizar)
dfListings['Municipality'] = dfListings['City']  # Substitui City por Municipality
dfListings['Municipality_norm'] = dfListings['Municipality'].apply(normalizar)
dfListings = dfListings.drop(columns=['City'])

# 💾 Exportar CSV final tratado
dfListings.to_csv(r'dados\dados tradados/listingsTratado.csv', index=False)


#### rendasm2

In [170]:
dfRendas = pd.DataFrame(rendas)

# 💶 Converter 'valor' para float
dfRendas['RentValue'] = pd.to_numeric(dfRendas['valor'], errors='coerce')

# 🆔 Extrair código do concelho (últimos 4 dígitos de geocod)
dfRendas['MunicipalityCode'] = dfRendas['geocod'].str[-4:]

# 🔤 Normalizar nome do município
dfRendas['MunicipalityNorm'] = dfRendas['geodsg'].apply(normalizar)

# 🔁 Juntar com concelhos apenas para validar matches
rendasMerged = dfRendas.merge(
    dfConcelhos[['MunicipalityCode', 'Municipality', 'Municipality_norm']],
    on='MunicipalityCode', how='left'
)

# ✅ Filtrar apenas matches válidos
rendasMatch = rendasMerged[rendasMerged['Municipality'].notna()].copy()

# 🔄 Substituir valores nulos pela média por concelho
rendasMatch['RentValue'] = rendasMatch.groupby('MunicipalityCode')['RentValue'].transform(
    lambda x: x.fillna(x.mean())
)

# 📊 Agregar média por concelho
rendasPorConcelho = (
    rendasMatch
    .groupby('MunicipalityCode')
    .agg({'RentValue': 'mean'})
    .reset_index()
    .rename(columns={'RentValue': 'RentValueMean'})
)

# 🧩 Adicionar Municipality e MunicipalityNorm
rendasPorConcelho = rendasPorConcelho.merge(
    dfConcelhos[['MunicipalityCode', 'Municipality', 'Municipality_norm']],
    on='MunicipalityCode', how='left'
)

dfRendas = pd.DataFrame(rendasPorConcelho)

# 💾 Exportar CSV final
dfRendas.to_csv(r'dados\dados tradados/rendasTratado.csv', index=False)


---

## 🔗 Plano de Integração de Esquemas

Após o processo de limpeza e normalização dos dados, foi definido o seguinte plano para integrar os diversos datasets, tendo em conta as diferenças de granularidade e estrutura dos mesmos.

### 🧩 Objetivo da Integração

Integrar os datasets a um **nível municipal (MunicipalityCode)** para permitir a análise cruzada entre:
- Preços das rendas por m² (`rendasm2`)
- Densidade populacional (`densidadePopulacional`)
- Densidade de alojamentos locais (`densidadeAlojamentosm2`)
- Preços dos imóveis (`portugal_listings`)
- Localização administrativa (`Areas_Concelhos_CAOP2019`)


### 🗂️ Granularidade dos Dados

| Dataset                | Granularidade Original | Estratégia de Uniformização       |
|------------------------|------------------------|-----------------------------------|
| `densidadeAlojamentos` | Freguesia              | Agrupar por concelho com a média  |
| `rendasm2`             | Freguesia              | Agrupar por concelho com a média  |
| `densidadePopulacional`| Concelho               | Já está no nível correto          |
| `portugal_listings`    | Freguesia              | Usar a coluna concelho            |
| `Areas_Concelhos_CAOP2019` | Concelho           | Base de referência administrativa |


### 🧱 Chaves e Mapeamentos

- A **chave de integração principal** será `MunicipalityCode`, extraída de `geocod`, `DICO`, ou derivada de `City`.
- Para garantir consistência, foi criada a coluna `Municipality_norm` (nomes normalizados de concelhos).
- Os nomes de cidades (`City`) serão normalizados e mapeados para os concelhos correspondentes manualmente ou via regra de decisão (ex: correspondência direta com `Municipality_norm`).


### 📋 Estratégia de Integração

1. **Densidade de Alojamentos**:  
   - Agregar por `MunicipalityCode` (média dos valores da freguesia).
   - Gerar dataset final com: `MunicipalityCode`, `DensityAlojMean` e `Municipality_norm`.

2. **Densidade Populacional**:  
   - Já vem por `MunicipalityCode`, apenas normalizar e confirmar correspondência.

3. **Rendas por m²**:  
   - Agregar por `MunicipalityCode` (média dos valores da freguesia).
   - Gerar dataset com: `MunicipalityCode`, `RentValueMean` e `Municipality_norm`.

4. **Listings**:  
   - Associar `Municipality_norm` a `Municipality_norm` da tabela `concelhosTratado.csv` e obter `MunicipalityCode`.
   - Calcular estatísticas agregadas (média de preço, contagem) por concelho.


### 🧠 Considerações Finais

- O dataset `concelhosTratado.csv` será a **tabela base** para todas as junções.
- As colunas `MunicipalityCode` e `Municipality_norm` serão usadas como **blocking keys**.
- Pode ser necessário fazer **resolução de identidade** entre nomes de cidades (`City`) e concelhos (`Municipality`) para garantir boas ligações entre listings e os restantes datasets.

---

## 🧱 Blocking Strategy & Similarity Metrics

Durante a resolução de identidade e planeamento da integração, é essencial preparar mecanismos que reduzam o custo computacional das comparações entre registos (blocking) e definam critérios objetivos para medir semelhança (similarity metrics).

### 📦 Blocking Strategy

**Objetivo:** Reduzir o número de comparações entre entidades de diferentes datasets, criando "blocos" onde a comparação faz sentido (ex: localidades com o mesmo prefixo ou região).

#### Estratégia adotada:
- Criar blocos com base no prefixo do nome normalizado (`Municipality_norm`) ou na `Region`.
- Apenas comparamos nomes dentro do mesmo bloco.

In [179]:
!pip install rapidfuzz

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.6 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.6 MB 1.7 MB/s eta 0:00:01
   ------------------------- -------------- 1.0/1.6 MB 1.9 MB/s eta 0:00:01
   -------------------------------- ------- 1.3/1.6 MB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 1.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [181]:
import pandas as pd
from rapidfuzz import fuzz

# 1️⃣ Carregar os datasets tratados
dfListings = pd.read_csv('dados/dados tradados/listingsTratado.csv')
dfRendas = pd.read_csv('dados/dados tradados/rendasTratado.csv')

# 2️⃣ Criar chave de blocking com prefixo (3 letras)
dfListings['block_key'] = dfListings['Municipality_norm'].str[:3]
dfRendas['block_key'] = dfRendas['Municipality_norm'].str[:3]

# 3️⃣ Gerar pares candidatos com base no bloco
candidatos = pd.merge(
    dfListings[['Municipality', 'Municipality_norm', 'block_key']],
    dfRendas[['Municipality', 'Municipality_norm', 'block_key']],
    on='block_key',
    suffixes=('_listings', '_rendas')
)

# 4️⃣ Calcular as 3 medidas de similaridade
candidatos['similaridade_token_sort'] = candidatos.apply(
    lambda row: fuzz.token_sort_ratio(row['Municipality_norm_listings'], row['Municipality_norm_rendas']) / 100,
    axis=1
)

candidatos['similaridade_levenshtein'] = candidatos.apply(
    lambda row: fuzz.ratio(row['Municipality_norm_listings'], row['Municipality_norm_rendas']) / 100,
    axis=1
)

candidatos['similaridade_token_set'] = candidatos.apply(
    lambda row: fuzz.token_set_ratio(row['Municipality_norm_listings'], row['Municipality_norm_rendas']) / 100,
    axis=1
)

# 5️⃣ Visualizar os pares com maior similaridade
top_pares = candidatos.sort_values(by='similaridade_token_sort', ascending=False)

# Mostrar apenas as colunas principais
display(top_pares[[
    'Municipality_listings', 'Municipality_rendas',
    'similaridade_token_sort',
    'similaridade_levenshtein',
    'similaridade_token_set'
]])


,Municipality_listings,Municipality_rendas,similaridade_token_sort,similaridade_levenshtein,similaridade_token_set
3,Valpaços,VALPAÇOS,1.00,1.00,1.00
576423,Valpaços,VALPAÇOS,1.00,1.00,1.00
576419,Alijó,ALIJÓ,1.00,1.00,1.00
576418,Valpaços,VALPAÇOS,1.00,1.00,1.00
576398,Alcácer do Sal,ALCÁCER DO SAL,1.00,1.00,1.00
...,...,...,...,...,...
35123,Peniche,PENALVA DO CASTELO,0.24,0.40,0.24
478530,Montalegre,MONDIM DE BASTO,0.24,0.32,0.24
196369,Peniche,PENALVA DO CASTELO,0.24,0.40,0.24
126329,Mondim de Basto,MONTALEGRE,0.24,0.32,0.24


## 📝 Conclusões e Trabalho Futuro

---